In [3]:
import pandas as pd

from serlo import SerloBot
from serlo.mysql import db


df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)
df.head()

,id,username,description,date,edits,reviews,comments,taxonomy
0,1,arekkas,"[[{""col"":24,""content"":""As MSc Computer Science...",2014-03-01 20:36:21,163.0,172.0,78.0,784.0
1,2,devuser,NULL,2014-03-01 20:36:21,2.0,1.0,4.0,3.0
2,4,Deleted,NULL,2014-03-01 20:36:21,161.0,187.0,7.0,17.0
3,6,Legacy,NULL,2014-03-01 20:36:21,6986.0,6988.0,0.0,1908.0
4,9,David,NULL,2014-03-01 20:36:32,0.0,0.0,1.0,0.0


In [6]:
def has_link(description):
    if description == None or description == "NULL":
        return False
    
    return "http://" in description or "https://" in description

df["has_link"] = df["description"].map(has_link)
df["date"] = df["date"].map(pd.to_datetime)
df.head()

,id,username,description,date,edits,reviews,comments,taxonomy,has_link
0,1,arekkas,"[[{""col"":24,""content"":""As MSc Computer Science...",2014-03-01 20:36:21,163.0,172.0,78.0,784.0,True
1,2,devuser,NULL,2014-03-01 20:36:21,2.0,1.0,4.0,3.0,False
2,4,Deleted,NULL,2014-03-01 20:36:21,161.0,187.0,7.0,17.0,False
3,6,Legacy,NULL,2014-03-01 20:36:21,6986.0,6988.0,0.0,1908.0,False
4,9,David,NULL,2014-03-01 20:36:32,0.0,0.0,1.0,0.0,False


In [15]:
df_delete = df[
    (df["edits"] + df["comments"] + df["reviews"] + df["taxonomy"] == 0) &
    df["has_link"] &
    (df["date"] < pd.to_datetime("2021-12-01"))
].copy()
df_delete.sort_values(["date"], inplace=True)
df_delete

,id,username,description,date,edits,reviews,comments,taxonomy,has_link
11495,145048,bestweightedblanket,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2019-09-28 15:01:22,0.0,0.0,0.0,0.0,True
11501,145056,Aida01,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2019-09-30 07:59:19,0.0,0.0,0.0,0.0,True
11521,145336,alldaypill122,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2019-10-03 11:28:41,0.0,0.0,0.0,0.0,True
11526,145351,fitnessdiet,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2019-10-04 12:05:08,0.0,0.0,0.0,0.0,True
11528,145377,nicholassmith,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2019-10-04 18:38:44,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...
22845,228130,ztorage,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2021-10-27 07:13:35,0.0,0.0,0.0,0.0,True
22848,228173,jasakontraktor,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2021-10-27 15:12:16,0.0,0.0,0.0,0.0,True
22851,228209,ruoumoyentuvn,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2021-10-27 18:26:14,0.0,0.0,0.0,0.0,True
22887,228454,dubaidesertsafarigroup,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2021-11-01 12:27:49,0.0,0.0,0.0,0.0,True
